In [ ]:
# Run conversion from KPF (Kwiver Packet Format) to DIVE JSON
# https://kwiver-diva.readthedocs.io/en/latest/kpf.html
# https://docs.google.com/document/d/14FGtZcpWSngh89lb8t7zFnIPQTeESoTZOEufqfXUqgI/edit#
from dive_server.serializers.kpf import meva
from dive_server.serializers import viame
import json

# Populate `./kpf` with the files listed below
activities_filename = './kpf/activities.yml'
geom_filename = './kpf/geom.yml'
types_filename = './kpf/types.yml'

# These files will be created in the same folder
output_json = 'kpf/result_dive_output.json'
output_csv = 'kpf/result_dive_output.csv'

activity_map = {}
actor_map = {}

# Populate track and actor maps
with open(activities_filename) as f:
    meva.deserialize_activities(f, activity_map, actor_map)
with open(geom_filename) as f:
    meva.deserialize_geom(f, actor_map)
with open(types_filename) as f:
    meva.deserialize_types(f, actor_map)

# Convert pydantic models to plain objects
trackmap = meva.parse_actor_map_to_tracks(actor_map)
plain_trackmap = {}
for track in trackmap.values():
    plain_trackmap[track.trackId] = track.dict(exclude_none=True)

# Export files
with open(output_json, "w") as out:
    out.write(json.dumps(plain_trackmap))
with open(output_csv, "w") as out:
    for line in viame.export_tracks_as_csv(plain_trackmap):
        out.write(line)

print(f'Found {len(trackmap.keys())} tracks and {len(actor_map.keys())} actors')